<a href="https://colab.research.google.com/github/ccspen21/greenland-fishery-nowcast-2025/blob/main/model_run_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from IPython.display import display

# Time Series of Fish Catch

In [ ]:
# Time Series of Fish Catch
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

df_clean["Time"] = df_clean["Year"].astype(str) + " " + df_clean["Quarter"]
df_clean["Time_Index"] = range(len(df_clean))

# Plot the time series
plt.figure(figsize=(12,6))
plt.plot(df_clean["Time_Index"], df_clean["Total_Catch"], marker='o', linestyle='-')

# Format x-axis: show only one label per year (use Q1 as the anchor)
plt.xticks(
    ticks=df_clean[df_clean["Quarter"] == "Q1"]["Time_Index"],
    labels=df_clean[df_clean["Quarter"] == "Q1"]["Year"],
    rotation=45
)

plt.xlabel("Year")
plt.ylabel("Total Catch (Tonnes)")
plt.title("Total Fish Catch Over Time")
plt.grid(True)
plt.tight_layout()
plt.savefig('total_fish_catch_over_time.png')

# Interaction Term

In [ ]:
# Create three-way interaction terms in each SST regional DataFrame
df_sst_west_clean["Melt_SST_Interaction_West"] = (
    df_sst_west_clean["Melt_Active_West"] *
    df_sst_west_clean["Melt_Index_West"] *
    df_sst_west_clean["Sea_Surface_Temp_C_West"]
)

df_sst_east_clean["Melt_SST_Interaction_East"] = (
    df_sst_east_clean["Melt_Active_East"] *
    df_sst_east_clean["Melt_Index_East"] *
    df_sst_east_clean["Sea_Surface_Temp_C_East"]
)

df_sst_south_clean["Melt_SST_Interaction_South"] = (
    df_sst_south_clean["Melt_Active_South"] *
    df_sst_south_clean["Melt_Index_South"] *
    df_sst_south_clean["Sea_Surface_Temp_C_South"]
)

# Merged Dataset

In [ ]:
# Ensure all 'Year' and 'Quarter' columns are of consistent type across DataFrames
def fix_keys(df):
    df["Year"] = df["Year"].astype(int)
    df["Quarter"] = df["Quarter"].astype(str).str.replace("quarter ", "Q").str.replace("Quarter ", "Q")
    return df

# Apply to all component DataFrames
df_clean = fix_keys(df_clean)
df_fish_clean = fix_keys(df_fish_clean)
df_foreign_clean = fix_keys(df_foreign_clean)
df_sst_west_clean = fix_keys(df_sst_west_clean)
df_sst_east_clean = fix_keys(df_sst_east_clean)
df_sst_south_clean = fix_keys(df_sst_south_clean)

# Start fresh from df_clean
df_merged_with_interactions = df_clean.copy()

# Merge standard right-hand-side variables
df_merged_with_interactions = df_merged_with_interactions.merge(df_fish_clean, on=["Year", "Quarter"], how="inner")

# Merge SST interaction terms
df_merged_with_interactions = df_merged_with_interactions.merge(
    df_sst_west_clean[["Year", "Quarter", "Melt_SST_Interaction_West"]],
    on=["Year", "Quarter"], how="inner"
).merge(
    df_sst_east_clean[["Year", "Quarter", "Melt_SST_Interaction_East"]],
    on=["Year", "Quarter"], how="inner"
).merge(
    df_sst_south_clean[["Year", "Quarter", "Melt_SST_Interaction_South"]],
    on=["Year", "Quarter"], how="inner"
)

# Merge foreign catch
df_merged_with_interactions = df_merged_with_interactions.merge(
    df_foreign_clean.drop(columns=["Unit"]),
    on=["Year", "Quarter"], how="left"
)

# Drop unnecessary columns
df_merged_with_interactions = df_merged_with_interactions.drop(
    columns=[col for col in df_merged_with_interactions.columns if "Unit" in col], errors="ignore"
)
df_merged_with_interactions = df_merged_with_interactions.drop(columns=["Time", "Time_Index"], errors="ignore")

# Order
df_merged_with_interactions["Quarter"] = pd.Categorical(
    df_merged_with_interactions["Quarter"], categories=["Q1", "Q2", "Q3", "Q4"], ordered=True
)
df_merged_with_interactions = df_merged_with_interactions.sort_values(by=["Year", "Quarter"]).reset_index(drop=True)

print("✅ Final merged dataset shape:", df_merged_with_interactions.shape)
display(df_merged_with_interactions.head())

In [ ]:
# Create df_model_with_interactions with Lagged Variables

# Step 1: Identify predictors to lag (exclude Total_Catch and lagged versions)
predictor_cols = [
    col for col in df_merged_with_interactions.columns
    if col not in ["Year", "Quarter", "Total_Catch"]
    and not col.endswith("_lag1")
    and not col.endswith("_lag2")
    and not col.endswith("_lag3")
    and not col.endswith("_lag4")
]

print("🔍 Predictors to lag:", predictor_cols)

# Step 2: Add lags 1–4 for each selected predictor
for col in predictor_cols:
    for lag in [1, 2, 3, 4]:
        df_merged_with_interactions[f"{col}_lag{lag}"] = df_merged_with_interactions[col].shift(lag)

# Step 3: Add lags 1–4 for Total_Catch
for lag in [1, 2, 3, 4]:
    df_merged_with_interactions[f"Total_Catch_lag{lag}"] = df_merged_with_interactions["Total_Catch"].shift(lag)

# Step 4: Drop original (non-lagged) predictors
df_model_with_interactions = df_merged_with_interactions.drop(columns=predictor_cols)

# Step 5: Drop rows with NA from lagging
df_model_with_interactions = df_model_with_interactions.dropna().reset_index(drop=True)

# Step 6: Define modeling dataset
y = df_model_with_interactions["Total_Catch"]
X = df_model_with_interactions.drop(columns=["Year", "Quarter", "Total_Catch"])

# ✅ Final shape check
print("✅ Clean setup with lags 1–4 — X shape:", X.shape, "y shape:", y.shape)

# Summary Statistics

In [ ]:
# Step 1: Identify main (non-lagged) variables
main_vars = [
    col for col in df_merged_with_interactions.columns
    if not col.endswith("_lag1")
    and not col.endswith("_lag2")
    and not col.endswith("_lag3")
    and not col.endswith("_lag4")
    and col not in ["Year", "Quarter"]
]

# Step 2: Subset numeric columns only (optional)
main_df = df_merged_with_interactions[main_vars].select_dtypes(include="number")

# Step 3: Generate descriptive statistics
summary_main = main_df.describe().T
summary_main["missing"] = main_df.isna().sum()
summary_main["skew"] = main_df.skew()
summary_main["kurtosis"] = main_df.kurt()

# Round for cleaner presentation
summary_main = summary_main.round(2)

# Step 4: Display
print("📊 Descriptive Statistics (Main Variables Only):")
display(summary_main)

# Correlation Matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

main_vars = [
    col for col in df_merged_with_interactions.columns
    if not col.endswith("_lag1") and not col.endswith("_lag2")
    and not col.endswith("_lag3") and not col.endswith("_lag4")
    and col not in ["Year", "Quarter"]
]

main_corr = df_merged_with_interactions[main_vars].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(main_corr, cmap="coolwarm", annot=True, fmt=".2f")
plt.title("Correlation Matrix of Main (Non-Lagged) Variables")
plt.tight_layout()
plt.savefig('correlation_matrix.png')

# Nowcasting Q1 2025

In [ ]:
# ✅ Use all available data up to Q4 2024
X_train_lasso_q1 = df_model_with_interactions[
    (df_model_with_interactions["Year"] < 2025)
][[
    "Total_Catch_lag4",
    "Melt_SST_Interaction_West_lag4",
    "Foreign_Catch_lag4",
    "Melt_SST_Interaction_East_lag1",
    "Fish_Export_Value_Million_Kr_lag2"
]]

y_train_lasso_q1 = df_model_with_interactions[
    (df_model_with_interactions["Year"] < 2025)
]["Total_Catch"]

# Step 2: Q1 2025 input for LASSO
X_nowcast_lasso_q1 = pd.DataFrame([{
    "Total_Catch_lag4": df_model_with_interactions.iloc[-4]["Total_Catch_lag4"],
    "Melt_SST_Interaction_West_lag4": df_model_with_interactions.iloc[-4]["Melt_SST_Interaction_West_lag4"],
    "Foreign_Catch_lag4": df_model_with_interactions.iloc[-4]["Foreign_Catch_lag4"],
    "Melt_SST_Interaction_East_lag1": df_model_with_interactions.iloc[-1]["Melt_SST_Interaction_East_lag1"],
    "Fish_Export_Value_Million_Kr_lag2": df_model_with_interactions.iloc[-4]["Fish_Export_Value_Million_Kr_lag2"]
}])

# Step 3: LASSO pipeline
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

lasso_pipeline_q1 = make_pipeline(
    StandardScaler(),
    Lasso(alpha=1.0, max_iter=10000)
)

lasso_pipeline_q1.fit(X_train_lasso_q1, y_train_lasso_q1)

# Step 4: Predict
y_pred_lasso_q1 = lasso_pipeline_q1.predict(X_nowcast_lasso_q1)[0]
print(f"📈 LASSO Nowcast for Q1 2025: {round(y_pred_lasso_q1):,.0f} tons")

# Graph of Nowcast

In [ ]:
import matplotlib.pyplot as plt

# Rebuild time axis if needed
df_clean["Time"] = df_clean["Year"].astype(str) + " " + df_clean["Quarter"]
df_clean["Time_Index"] = range(len(df_clean))

# Define the index and value for Q4 2024 (last actual) and Q1 2025 (nowcast)
last_actual_index = df_clean["Time_Index"].max()
last_actual_value = df_clean["Total_Catch"].iloc[-1]

nowcast_index = last_actual_index + 1
nowcast_value = y_pred_lasso_q1

# Plot historical series
plt.figure(figsize=(12, 6))
plt.plot(df_clean["Time_Index"], df_clean["Total_Catch"], marker='o', linestyle='-', label="Historical")

# Add red dot for nowcast
plt.plot(nowcast_index, nowcast_value, 'ro', label="LASSO Nowcast (Q1 2025)", markersize=8)

# Add red connecting line from Q4 2024 to Q1 2025
plt.plot([last_actual_index, nowcast_index], [last_actual_value, nowcast_value], color='red', linestyle='-', linewidth=2)

# Update x-ticks to include 2025
xticks = list(df_clean[df_clean["Quarter"] == "Q1"]["Time_Index"])
xticks_labels = list(df_clean[df_clean["Quarter"] == "Q1"]["Year"].astype(str))
xticks.append(nowcast_index)
xticks_labels.append("2025")

plt.xticks(ticks=xticks, labels=xticks_labels, rotation=45)

plt.text(
    nowcast_index + 0.2,          # shift right
    nowcast_value + 2000,         # shift upward
    f"{round(nowcast_value):,}",
    color="red",
    fontsize=9.5,
)

plt.axvspan(nowcast_index - 0.1, nowcast_index + 0.1, color='red', alpha=0.1)

plt.axvline(nowcast_index, color='gray', linestyle='--', linewidth=1)

# Labels and formatting
plt.xlabel("Year")
plt.ylabel("Total Catch (Tonnes)")
plt.title("Total Fish Catch Over Time with LASSO Nowcast")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig('total_fish_catch_with_nowcast.png')

# Capelin Dummy - Example of Policy Intervention

Note on Capelin Dummy
The Capelin_Open dummy variable is introduced here to model the impact of policy interventions (e.g., fishery closures in 2019, 2020, 2024, and 2025). This variable is used specifically for nowcasting in this notebook and is not included in the LASSO model fitting performed in model_fitting_diagnostics.ipynb. This allows us to isolate the effect of policy shocks in our nowcast analysis.


In [ ]:
# Step 1: Start with a fresh copy
df_with_capelin_dummy = df_model_with_interactions.copy()

# Step 2: Add Capelin_Open dummy variable
df_with_capelin_dummy["Capelin_Open"] = df_with_capelin_dummy["Year"].apply(
    lambda x: 0 if x in [2019, 2020, 2024, 2025] else 1
)

# Step 3: Identify predictors to lag (excluding identifiers, Capelin_Open, and already-lagged variables)
predictor_cols = [
    col for col in df_with_capelin_dummy.columns
    if col not in ["Year", "Quarter", "Total_Catch", "Capelin_Open"]
    and not col.endswith("_lag1")
    and not col.endswith("_lag2")
    and not col.endswith("_lag3")
    and not col.endswith("_lag4")
]

# Step 4: Add lags (1–4) for all predictors
for col in predictor_cols:
    for lag in [1, 2, 3, 4]:
        df_with_capelin_dummy[f"{col}_lag{lag}"] = df_with_capelin_dummy[col].shift(lag)

# Step 5: Add lags for Total Catch
for lag in [1, 2, 3, 4]:
    df_with_capelin_dummy[f"Total_Catch_lag{lag}"] = df_with_capelin_dummy["Total_Catch"].shift(lag)

# Step 6: Drop original non-lagged predictors (keep Capelin_Open)
df_model_capelin = df_with_capelin_dummy.drop(columns=predictor_cols)

# Step 7: Drop rows with NA caused by lagging
df_model_capelin = df_model_capelin.dropna().reset_index(drop=True)

# Step 8: Define target and features
y_capelin = df_model_capelin["Total_Catch"]
X_capelin = df_model_capelin.drop(columns=["Year", "Quarter", "Total_Catch"])

# ✅ Final check
print("✅ X_capelin shape:", X_capelin.shape)
print("✅ y_capelin shape:", y_capelin.shape)
print("✅ Final predictors:", X_capelin.columns.tolist())

# Nowcast with Capelin Dummy

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Step 1: Training data (up to Q4 2024)
train_q1_2025_binary = df_model_capelin[
 "Total_Catch_lag4",
    "Foreign_Catch_lag4",
    "Melt_SST_Interaction_West_lag4",
    "Melt_SST_Interaction_East_lag1",
    "Fish_Export_Value_Million_Kr_lag2",
    "Capelin_Open"
]]
y_train_q1_2025_binary = train_q1_2025_binary["Total_Catch"]

print("✅ Training set size with Capelin dummy:", X_train_q1_2025_binary.shape)

# Step 2: Input row for Q1 2025
latest_row_q4_binary = df_model_capelin[
    (df_model_capelin["Year"] == 2024) &
    (df_model_capelin["Quarter"] == "Q4")
].iloc[0]

X_nowcast_q1_2025_binary = pd.DataFrame([{
    "Total_Catch_lag4": df_model_capelin.iloc[-4]["Total_Catch_lag4"],
    "Foreign_Catch_lag4": df_model_capelin.iloc[-4]["Foreign_Catch_lag4"],
    "Melt_SST_Interaction_West_lag4": df_model_capelin.iloc[-4]["Melt_SST_Interaction_West_lag4"],
    "Melt_SST_Interaction_East_lag1": df_model_capelin.iloc[-1]["Melt_SST_Interaction_East_lag1"],
    "Fish_Export_Value_Million_Kr_lag2": df_model_capelin.iloc[-4]["Fish_Export_Value_Million_Kr_lag2"],
    "Capelin_Open": 0  # manually set to 0 because 2025 is closed
}])

print("✅ Nowcast input for Q1 2025 with Capelin dummy:")
display(X_nowcast_q1_2025_binary)

# Step 3: Lasso model pipeline
lasso_pipeline_q1_binary = make_pipeline(
    StandardScaler(),
    Lasso(alpha=1.0, max_iter=50000, random_state=42)  # You can tune alpha
)

# Step 4: Fit and predict
lasso_pipeline_q1_binary.fit(X_train_q1_2025_binary, y_train_q1_2025_binary)
y_pred_q1_2025_binary = lasso_pipeline_q1_binary.predict(X_nowcast_q1_2025_binary)[0]

# Step 5: Output result
print(f"📈 📊 Lasso Nowcast for Q1 2025 (with Capelin dummy): {round(y_pred_q1_2025_binary):,.0f} tons")

## Graph Comparison of Nowcasts

In [ ]:
import matplotlib.pyplot as plt

# Rebuild time axis if needed
df_clean["Time"] = df_clean["Year"].astype(str) + " " + df_clean["Quarter"]
df_clean["Time_Index"] = range(len(df_clean))

# Final observed point (Q4 2024)
last_actual_index = df_clean["Time_Index"].max()
last_actual_value = df_clean["Total_Catch"].iloc[-1]

# Prediction point (Q1 2025)
nowcast_index = last_actual_index + 1
nowcast_std = y_pred_lasso_q1                 # Standard LASSO (no Capelin)
nowcast_binary = y_pred_q1_2025_binary        # LASSO with Capelin dummy

# Plot historical data
plt.figure(figsize=(12, 6))
plt.plot(df_clean["Time_Index"], df_clean["Total_Catch"], marker='o', linestyle='-', label="Historical")

# Plot both nowcasts as dots
plt.plot(nowcast_index, nowcast_std, 'ro', label="LASSO Nowcast (No Capelin)", markersize=8)
plt.plot(nowcast_index, nowcast_binary, 'go', label="LASSO Nowcast (With Capelin)", markersize=8)

# Connect last actual to each forecast
plt.plot([last_actual_index, nowcast_index], [last_actual_value, nowcast_std],
         color='red', linestyle='-', linewidth=2)
plt.plot([last_actual_index, nowcast_index], [last_actual_value, nowcast_binary],
         color='green', linestyle='-', linewidth=2)

# Annotate both nowcast values
plt.text(nowcast_index + 0.2, nowcast_std + 2000,
         f"{round(nowcast_std):,}", color="red", fontsize=9.5)

plt.text(nowcast_index + 0.2, nowcast_binary + 2000,
         f"{round(nowcast_binary):,}", color="green", fontsize=9.5)

# Forecast area shading and vertical marker
plt.axvspan(nowcast_index - 0.1, nowcast_index + 0.1, color='gray', alpha=0.1)
plt.axvline(nowcast_index, color='gray', linestyle='--', linewidth=1)

# X-axis setup
xticks = list(df_clean[df_clean["Quarter"] == "Q1"]["Time_Index"])
xticks_labels = list(df_clean[df_clean["Quarter"] == "Q1"]["Year"].astype(str))
xticks.append(nowcast_index)
xticks_labels.append("2025")
plt.xticks(ticks=xticks, labels=xticks_labels, rotation=45)

# Labels and formatting
plt.xlabel("Year")
plt.ylabel("Total Catch (Tonnes)")
plt.title("Total Fish Catch Over Time: LASSO Nowcast With and Without Capelin Dummy")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig('nowcast_comparison.png')

# Actual vs. Predicted Plot for Backtests

In [ ]:
import matplotlib.pyplot as plt

# Define the quarters and values
quarters = ["Q1 2024", "Q2 2024", "Q3 2024", "Q4 2024"]
actual_values = [actual_q1, actual_q2, actual_q3, actual_q4]
predicted_values = [
    y_pred_q1_2024_lasso,
    y_pred_q2_2024_lasso,
    y_pred_q3_2024_lasso,
    y_pred_q4_2024_lasso
]

# Calculate y-axis padding
all_values = actual_values + predicted_values
y_min = min(all_values) * 0.9  # 10% below
y_max = max(all_values) * 1.1  # 10% above

# Plot
plt.figure(figsize=(10, 6))
plt.plot(quarters, actual_values, label="Actual", marker='o', linewidth=2)
plt.plot(quarters, predicted_values, label="LASSO Prediction", marker='o', linewidth=2)
plt.ylim(y_min, y_max)
plt.title("📊 Actual vs. Predicted Fish Catch (LASSO, Q1–Q4 2024)")
plt.xlabel("Quarter")
plt.ylabel("Total_Catch (tons)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig('actual_vs_predicted.png')